# Clasificación de emergencias mediante OPEN AI
En este notebook se muestra el proceso de análisis del archivo .json generado durante el análisis de sonido, con las características acústicas y datos principales del audio de voz. 
En el proceso se utiliza la tecnología de OPEN AI, para hacer un llamado a Chat GTP 4o, en el que se presenta la información del archivo de audio, y se solicita como resultado la identificación de una emergencia y su clasificación.

In [15]:
from openai import OpenAI
import json
import os
from dotenv import load_dotenv

# Caragar archivo .env con el api_key
load_dotenv()

# Obtener la clave API 
api_key = os.getenv("api_key_openai")
if not api_key:
    raise ValueError("The 'api_key' environment variable is not defined in the .env file.")

# Configura tu clave API de OpenAI aquí
client=OpenAI(api_key=api_key)

# Función para utilizar GPT-4 y analizar el texto
def analizar_emergencia_con_gpt(data):
    
    try:
        # Abrir el archivo JSON y cargar los datos
        with open(archivo_json, 'r') as f:
            data = json.load(f)  # Cargar el archivo JSON como un diccionario
    except json.JSONDecodeError:
        # Si el archivo JSON no es válido o está mal formado
        print("Error al leer el archivo JSON. Asegúrate de que el archivo esté correctamente formado.")
        return []  # Devolver una lista vacía en caso de error
    except FileNotFoundError:
        # Si el archivo no es encontrado
        print(f"El archivo {archivo_json} no fue encontrado.")
        return []  # Devolver una lista vacía en caso de error

    try:
        # Llamada a la API de OpenAI para analizar el mensaje usando el endpoint de chat

        response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "Eres un asistente que clasifica mensajes de emergencia para un chatbot de seguridad en minería subterránea.\n\nPor ejemplo:\n- {\"nombre_audio\":\"voice_message.wav\",\"texto\":\" En el revaje de la rampa 5 hay un incendio para donde corremos, dinos r\\u00e1pido hacia donde corremos.\",\"idioma_audio\":\"es\",\"palabras\":18,\"duracion\":5.44,\"tempo\":95.703125,\"zcr\":0.0752368684,\"energia\":1135.146484375,\"picos_env\":2.444185257,\"estres\":1} respuesta: {emergencia:1, tipo:incendio}\n\nOtro ejemplo:\n- {\"nombre_audio\":\"voice_message.wav\",\"texto\":\" Est\\u00e1 saliendo humo de un tubo. Eso es normal, ayuda.\",\"idioma_audio\":\"es\",\"palabras\":10,\"duracion\":5.6135147392,\"tempo\":135.9991776316,\"zcr\":0.0468205223,\"energia\":365.2601318359,\"picos_env\":1.7363413572,\"estres\":0} respuesta: {emergencia:1, tipo:incendio}\n\nOtro ejemplo:\n- {\"nombre_audio\":\"voice_message.wav\",\"texto\":\" Hay un compañero tirado en el rebaje, que hago.\",\"idioma_audio\":\"es\",\"palabras\":9,\"duracion\":5.6135147392,\"tempo\":135.9991776316,\"zcr\":0.0468205223,\"energia\":365.2601318359,\"picos_env\":1.7363413572,\"estres\":1} respuesta: {emergencia:1, tipo: salud general}\n\nOtro ejemplo:\n- {\"nombre_audio\":\"voice_message.wav\",\"texto\":\" Necesito encontrar un baño, es una emergencia.\",\"idioma_audio\":\"es\",\"palabras\":7,\"duracion\":5.6135147392,\"tempo\":135.9991776316,\"zcr\":0.0468205223,\"energia\":365.2601318359,\"picos_env\":1.7363413572,\"estres\":1} respuesta: {emergencia:0, tipo: no_emergencia, idioma:es}\n\nOtro ejemplo:\n- {\"nombre_audio\":\"voice_message.wav\",\"texto\":\" Las bombas centrales de agua se apagaron.\",\"idioma_audio\":\"es\",\"palabras\":7,\"duracion\":5.6135147392,\"tempo\":135.9991776316,\"zcr\":0.0468205223,\"energia\":365.2601318359,\"picos_env\":1.7363413572,\"estres\":0} respuesta: {emergencia:1, tipo: inundación, idioma:es}\n\nOtro ejemplo:\n- {\"nombre_audio\":\"voice_message.wav\",\"texto\":\"Que hago si hay fuego en el taller y sale mucho humo.\",\"idioma_audio\":\"es\",\"palabras\":7,\"duracion\":5.6135147392,\"tempo\":135.9991776316,\"zcr\":0.0468205223,\"energia\":365.2601318359,\"picos_env\":1.7363413572,\"estres\":0} respuesta: {emergencia:0, tipo: no_emergencia, idioma:es}" 
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": f"Clasifica el siguiente mensaje de voz como una emergencia o no emergencia. Si es una emergencia, identifica el tipo (por ejemplo, incendio, inundación, caída de roca, enfermedad general, etc. Además, incluye el idioma del texto) {data}"
        }
            ]
            }
        ],
        response_format={
            "type": "text"
        },
        temperature=1,
        max_completion_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
        )

        # Obtener la respuesta generada por GPT-4
        respuesta = response.choices[0].message.content  # Extraer la respuesta de la API
        return respuesta
    except Exception as e:
        print(f"Error al procesar la API de OpenAI: {e}")
        return ""

# Función para procesar el archivo JSON y obtener resultados
def procesar_archivo_json(archivo_json):
    try:
        with open(archivo_json, 'r') as f:
            data = json.load(f)  # Cargar el JSON como un diccionario único
    except json.JSONDecodeError:
        print("Error al leer el archivo JSON. Asegúrate de que el archivo esté correctamente formado.")
        return []
    except FileNotFoundError:
        print(f"El archivo {archivo_json} no fue encontrado.")
        return []

    # Analizar el texto usando GPT-4
    resultado_gpt = analizar_emergencia_con_gpt(data)
    return resultado_gpt

# Ejecutar la función con el archivo JSON
archivo_json = 'audios_data.json'  # Ruta del archivo JSON
respuesta = procesar_archivo_json(archivo_json)

# Guardar los resultados en una variable
resultado_final = respuesta


Finalmente, se imprime el diccionario que almacena el resultado de la clasificación

In [16]:
resultado_final

'{emergencia: 1, tipo: incendio, idioma: es}'

In [ ]:
def clasificar_emergencia(archivo_json):
    from openai import OpenAI
    import json
    import os
    from dotenv import load_dotenv
    # Cargar archivo .env con el api_key
    load_dotenv()

    # Obtener la clave API
    api_key = os.getenv("api_key_openai")
    if not api_key:
        raise ValueError("The 'api_key' environment variable is not defined in the .env file.")

    # Configura tu clave API de OpenAI aquí
    client = OpenAI(api_key=api_key)

    try:
        # Abrir el archivo JSON y cargar los datos
        with open(archivo_json, 'r') as f:
            data = json.load(f)  # Cargar el archivo JSON como un diccionario
    except json.JSONDecodeError:
        # Si el archivo JSON no es válido o está mal formado
        print("Error al leer el archivo JSON. Asegúrate de que el archivo esté correctamente formado.")
        return []
    except FileNotFoundError:
        # Si el archivo no es encontrado
        print(f"El archivo {archivo_json} no fue encontrado.")
        return []

    try:
        # Llamada a la API de OpenAI para analizar el mensaje usando el endpoint de chat
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": [
                        {
                            "type": "text",
                            "text": "Eres un asistente que clasifica mensajes de emergencia para un chatbot de seguridad en minería subterránea.\n\nPor ejemplo:\n- {\"nombre_audio\":\"voice_message.wav\",\"texto\":\" En el revaje de la rampa 5 hay un incendio para donde corremos, dinos r\\u00e1pido hacia donde corremos.\",\"idioma_audio\":\"es\",\"palabras\":18,\"duracion\":5.44,\"tempo\":95.703125,\"zcr\":0.0752368684,\"energia\":1135.146484375,\"picos_env\":2.444185257,\"estres\":1} respuesta: {emergencia:1, tipo:incendio}\n\nOtro ejemplo:\n- {\"nombre_audio\":\"voice_message.wav\",\"texto\":\" Est\\u00e1 saliendo humo de un tubo. Eso es normal, ayuda.\",\"idioma_audio\":\"es\",\"palabras\":10,\"duracion\":5.6135147392,\"tempo\":135.9991776316,\"zcr\":0.0468205223,\"energia\":365.2601318359,\"picos_env\":1.7363413572,\"estres\":0} respuesta: {emergencia:1, tipo:incendio}\n\nOtro ejemplo:\n- {\"nombre_audio\":\"voice_message.wav\",\"texto\":\" Hay un compañero tirado en el rebaje, que hago.\",\"idioma_audio\":\"es\",\"palabras\":9,\"duracion\":5.6135147392,\"tempo\":135.9991776316,\"zcr\":0.0468205223,\"energia\":365.2601318359,\"picos_env\":1.7363413572,\"estres\":1} respuesta: {emergencia:1, tipo: salud general}\n\nOtro ejemplo:\n- {\"nombre_audio\":\"voice_message.wav\",\"texto\":\" Necesito encontrar un baño, es una emergencia.\",\"idioma_audio\":\"es\",\"palabras\":7,\"duracion\":5.6135147392,\"tempo\":135.9991776316,\"zcr\":0.0468205223,\"energia\":365.2601318359,\"picos_env\":1.7363413572,\"estres\":1} respuesta: {emergencia:0, tipo: no_emergencia, idioma:es}\n\nOtro ejemplo:\n- {\"nombre_audio\":\"voice_message.wav\",\"texto\":\" Las bombas centrales de agua se apagaron.\",\"idioma_audio\":\"es\",\"palabras\":7,\"duracion\":5.6135147392,\"tempo\":135.9991776316,\"zcr\":0.0468205223,\"energia\":365.2601318359,\"picos_env\":1.7363413572,\"estres\":0} respuesta: {emergencia:1, tipo: inundación, idioma:es}\n\nOtro ejemplo:\n- {\"nombre_audio\":\"voice_message.wav\",\"texto\":\"Que hago si hay fuego en el taller y sale mucho humo.\",\"idioma_audio\":\"es\",\"palabras\":7,\"duracion\":5.6135147392,\"tempo\":135.9991776316,\"zcr\":0.0468205223,\"energia\":365.2601318359,\"picos_env\":1.7363413572,\"estres\":0} respuesta: {emergencia:0, tipo: no_emergencia, idioma:es}"
                        }
                    ]
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": f"Clasifica el siguiente mensaje de voz como una emergencia o no emergencia. Si es una emergencia, identifica el tipo (por ejemplo, incendio, inundación, caída de roca, enfermedad general, etc. Además, incluye el idioma del texto) {data}"
                        }
                    ]
                }
            ],
            response_format={"type": "text"},
            temperature=1,
            max_completion_tokens=2048,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )

        # Obtener la respuesta generada por GPT-4
        respuesta = response.choices[0].message.content  # Extraer la respuesta de la API
        return respuesta
    except Exception as e:
        print(f"Error al procesar la API de OpenAI: {e}")
        return ""

In [17]:
from funciones import clasificar_emergencia

# Llamar la función con la ruta del archivo JSON
archivo_json = 'audios_data.json'  # Ruta del archivo JSON
respuesta = clasificar_emergencia(archivo_json)


In [18]:
respuesta

'{emergencia:1, tipo:incendio, idioma:es}'